In [141]:
import numpy as np
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pint import UnitRegistry

## constants

- source: fluid properties https://www.climate-policy-watcher.org/wastewater-sludge/physical-and-biological-properties.html

In [176]:
u = UnitRegistry()
kg = u.kilogram
m = u.meter 
s = u.second


In [236]:
def constants(rho, mu,  D, h, beta, override=False, perc_grade=0.015, alpha=2, g=9.8):

    # -----> to calculate pipe length, L 

    # max grade, slope s
    perc_grade = 0.015 

    # pipe horz distance travelled, x 
    x = h / perc_grade

    # angle required, theta 
    # TODO check if this is h/x or x/h 
    theta = np.rad2deg(np.arctan(x/h))

    # pipe length, L
    L = x / np.sin(theta) # m
    
    c = {
        # --- fluid ---------------> 
        # density of primary sludge
        "rho": rho * kg / (m**3),
        # dynamic viscosity
        "mu ": mu * kg / (m*s),
        # ---- infra ------------->
        # pipe diameter 
        "D": D * m,  
        # lift height, h 
        "h": h * m,
        # tank height, beta 
        "beta": beta * m,
        # total height dif
        "H": h + beta,
        # total horz distance 
        "x ": x,
        # angle 
        "theta": theta,
        # pipe length 
        "L": L,
        # ---- physics ----------->
        "g": g * m / (s**2),
        "alpha": alpha
        
    }

    if override:
        # use values from the Cengel example 8.1
        c["rho"] = 998 * kg / (m**3)
        c["mu"] = 0.00102 * kg / (m*s)
        c["D"] = 0.06 * m  
        c["L"] = 65 * m 
        c["H"] = 2.20 * m 

    return c



In [191]:
# fluid properties

# density of primary sludge => 1.0 -> 1.03 g/cm3 =>=> 1000 kg/m3 -> 1030 kg/m3
rho = 998 * kg / (m**3)

# dynamic viscosity => 0.5 and 2 Pa*s  ~ kg/(m*s)
mu = 0.00102 * kg / (m*s)

fluid_prop = {
    "rho": rho,
    "mu ": mu,
}
fluid_prop

{'rho': 998.0 <Unit('kilogram / meter ** 3')>,
 'mu ': 0.00102 <Unit('kilogram / meter / second')>}

In [193]:
# infrastructure properties 

# pipe diameter, D 
D = 0.06 * m  

# lift height, h 
h = 1.20 * m 

# tank height, beta 
beta =  1 * m 

H = h + beta
H = 2.20 * m 

# -----> to calculate pipe length 

# max grade, slope s
perc_grade = 0.015 

# pipe horz distance travelled, x 
x = h / perc_grade

# angle required, theta # TODO check if this is h/x or x/h 
theta = np.rad2deg(np.arctan(x/h))

# pipe length, L
L = x / np.sin(theta) # m


L = 65 * m # m, from book example 

infra_prop = {
    "lift height, h": h,
    "horz distance, x ": x,
    "theta": theta,
    "pipe length": L,
    "total height dif": H
}



print(infra_prop)


{'lift height, h': <Quantity(1.2, 'meter')>, 'horz distance, x ': <Quantity(80.0, 'meter')>, 'theta': <Quantity(89.1406278, 'degree')>, 'pipe length': <Quantity(65, 'meter')>, 'total height dif': <Quantity(2.2, 'meter')>}


In [194]:
mu * L

0.0663 <Unit('kilogram / second')>

In [181]:
# other physics properties?

# alpha, constant for fully developed flow at the outlet of a hose 
alpha = 2

# gravitational acceleration 
g = 9.8 * m / (s**2)

In [205]:
a = 1
b = (64 * mu * L ) / (rho * D**2 * alpha )
c =  -(2 * g * H) / (alpha)

coeff = {
    "a": a,
    "b": b,
    "c": c
}

print(coeff)

{'a': 1, 'b': <Quantity(0.590514362, 'meter / second')>, 'c': <Quantity(-21.56, 'meter ** 2 / second ** 2')>}


## plot

In [208]:
x =  np.linspace(-10, 10, 100)  * m / s

y = (a * x**2) + (b * x) + c

y[0]


72.53485637942552 <Unit('meter ** 2 / second ** 2')>

In [209]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x.magnitude,
    y=y.magnitude, 
    mode='lines+markers',
))

fig.update_layout(title='Examine Quad - Solving for V',
                   xaxis_title='x ',
                   yaxis_title='y')

In [231]:
def solve_quad(a, b, c):
    # a = a
    # b = b.magnitude
    # c = c.magnitude
    print(a,b,c)
    d = b ** 2 - 4 * a * c  # this part is called the discriminant

    if d < 0:
        print("The equation has no real solutions")
    elif d == 0:
        x = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        print(f"The equation has one solution: {x} ")
        return x
    else:
        x1 = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        x2 = (-b - np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        print(f"The equation has two solutions: {x1} or {x2}")
        return (x1,x2)
    

In [232]:
vel_result = solve_quad(a, b, c)

vel_result 

1 0.5905143620574482 meter / second -21.560000000000002 meter ** 2 / second ** 2
The equation has two solutions: 4.357395482017275 meter / second or -4.947909844074723 meter / second


(4.357395482017275 <Unit('meter / second')>,
 -4.947909844074723 <Unit('meter / second')>)

In [233]:
vel = vel_result[0]

## volume flow rate

In [234]:
# vfr , volume flow rate, m3/s
vfr = (vel* math.pi * D**2) / 4
vfr

0.012320245471581745 <Unit('meter ** 3 / second')>

## work from the pump 

In [235]:
w_pump = (vfr * 64 * mu * L * vel) / 2 * D**2 * g
w_pump

0.004018263596188509 <Unit('kilogram * meter ** 7 / second ** 5')>